In [1]:
import scanpy as sc
import numpy as np
import csv
import pickle
import matplotlib
import math
from itertools import combinations
import itertools
import pandas as pd
import matplotlib
from pyvis.network import Network
import networkx as nx
from networkx.algorithms import bipartite
from cdlib import algorithms
from cdlib import NodeClustering
import gseapy as gp
import altair as alt
import sys
import os
sys.path.append("/Users/victoriagao/Documents/MSc/Schwartz_lab/altair-themes/")
if True:  # In order to bypass isort when saving
    import altairThemes

spot_diameter = 89.43 #pixels

Note: to be able to use all crisp methods, you need to install some additional packages:  {'bayanpy', 'graph_tool', 'wurlitzer', 'infomap'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw', 'pyclustering'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}


In [4]:
# GSEA performed on preranked gene sets, ranked by log odds ratio

# Read in the whole log odds table
whole_log_odds_df = pd.read_csv('/Users/victoriagao/local_docs/NEST/output/LR_log_odds/BIG_log_odds.csv')



In [5]:
whole_log_odds_df

,L-R Pair,region1 VS 2,region1 VS 3,region1 VS 4,region1 VS 5,region1 VS 6,region1 VS 7,region1 VS 8,region1 VS 9,region1 VS 10,...,region16 VS 17,region16 VS 18,region16 VS 19,region16 VS 20,region17 VS 18,region17 VS 19,region17 VS 20,region18 VS 19,region18 VS 20,region19 VS 20
0,LGALS3-NPTN,2.189256,2.330756,2.698481,inf,1.550597,inf,inf,inf,inf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FN1-RPSA,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PTPRF-RACK1,-inf,NaN,-inf,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INS-HLA,-inf,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TIMP1-LRP1,-inf,NaN,-inf,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,BMP4-BMPR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,BMP2-ENG,-inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,HMGB1-AGTRAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,TGFB1-SDC2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Define a list of gene sets you want to use
gene_sets = ['MSigDB_Oncogenic_Signatures', 'MSigDB_Hallmark_2020']

output_folder = "/Users/victoriagao/local_docs/NEST/output/gsea_results/"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# split_df = LR_rank_df['L-R Pair'].str.split('-', expand=True)
#     split_df = split_df.iloc[:, :2]
#     split_df.columns = ['L', 'R']

In [17]:

# Get the L-R pair information from the first column of master_log_odds_df
LR_pairs_info = whole_log_odds_df.iloc[:, 0]

# Iterate through each column of master_log_odds_df, excluding the first column
for column_name, log_odds_values in whole_log_odds_df.iloc[:, 1:].items():
    try:
    # Combine the L-R pair information with the log odds values
        LR_pair_data = pd.DataFrame({'L-R Pair': LR_pairs_info, 'Log Odds': log_odds_values})
        
        # Rank the LR pairs based on their log odds ratio value
        ranked_LR = LR_pair_data.sort_values(by='Log Odds', ascending=False)
        ranking = LR_pair_data['Log Odds'].rank(ascending=False)
        LR_rank_df = pd.DataFrame({'L-R Pair': ranked_LR['L-R Pair'], 'Ranking': ranking.values})
        
        # Split the L-R Pair and create a new DataFrame with separate rows for L and R
        split_df = LR_rank_df['L-R Pair'].str.split('-', expand=True)
        split_df = split_df.iloc[:, :2]
        split_df.columns = ['L', 'R']
        
        # Create a new DataFrame with concatenated "L" and "R" values and "Ranking"
        gene_rank_df = split_df[['L', 'R']].stack().reset_index(level=1, drop=True).reset_index()
        gene_rank_df = pd.DataFrame(gene_rank_df.iloc[:,1])
        gene_rank_df.columns = ['Gene']
        gene_rank_df['Ranking'] = LR_rank_df['Ranking'].repeat(2).reset_index(drop=True)
        
        # Reorder columns
        gene_rank_df = gene_rank_df[['Gene', 'Ranking']]

        # Run GSEAPy for the current column
        pre_res = gp.prerank(rnk=gene_rank_df,
                            gene_sets=gene_sets,
                            threads=4,
                            min_size=1,
                            max_size=1000,
                            permutation_num=1000,
                            outdir=None,
                            seed=6,
                            verbose=True)
        
        # Sort the GSEA results by Normalized Enrichment Score (NES)
        pre_res = pre_res.res2d.sort_values(by='NES', ascending=False)
        print("success for",column_name)
        # Save the GSEA results to a file with a descriptive name
        result_file_name = f"gsea_results_{column_name}.csv"
        result_file_path = os.path.join(output_folder, result_file_name)
        pre_res.to_csv(result_file_path, index=False)
    except Exception as e:
        print(f"Error for column {column_name}: {e}")

    


2023-09-27 16:28:14,291 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!
2023-09-27 16:28:14,296 [WARNING] Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!
2023-09-27 16:28:14,303 [WARNING] Duplicated values found in preranked stats: 95.35% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2023-09-27 16:28:14,304 [INFO] Parsing data files for GSEA.............................
2023-09-27 16:28:14,304 [INFO] Enrichr library gene sets already downloaded in: /Users/victoriagao/.cache/gseapy, use local file
2023-09-27 16:28:14,320 [INFO] Enrichr library gene sets already downloaded in: /Users/victoriagao/.cache/gseapy, use local file
2023-09-27 16:28:14,331 [INFO] 0053 gene_sets have been filtered out when max_size=1000 and min_size=1
2023-09-27 16:28:14,331 [INFO] 0186 gene_sets used for further statistical testing.....
2023-09-27 16:28:14,331 [INFO] Start to run

success for region1 VS 2
success for region1 VS 3
success for region1 VS 4
success for region1 VS 5


2023-09-27 16:28:14,615 [INFO] 0120 gene_sets have been filtered out when max_size=1000 and min_size=1
2023-09-27 16:28:14,615 [INFO] 0119 gene_sets used for further statistical testing.....
2023-09-27 16:28:14,616 [INFO] Start to run GSEA...Might take a while..................
2023-09-27 16:28:14,661 [INFO] Congratulations. GSEApy runs successfully................

2023-09-27 16:28:14,664 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!
2023-09-27 16:28:14,667 [WARNING] Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!
2023-09-27 16:28:14,668 [WARNING] Duplicated values found in preranked stats: 80.00% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2023-09-27 16:28:14,668 [INFO] Parsing data files for GSEA.............................
2023-09-27 16:28:14,669 [INFO] Enrichr library gene sets already downloaded in: /Users/victoriagao/.cache/gseapy, use local

success for region1 VS 6
success for region1 VS 7
success for region1 VS 8
success for region1 VS 9
success for region1 VS 10
success for region1 VS 11


2023-09-27 16:28:14,873 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!
2023-09-27 16:28:14,876 [WARNING] Duplicated values found in preranked stats: 85.71% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2023-09-27 16:28:14,877 [INFO] Parsing data files for GSEA.............................
2023-09-27 16:28:14,877 [INFO] Enrichr library gene sets already downloaded in: /Users/victoriagao/.cache/gseapy, use local file
2023-09-27 16:28:14,882 [INFO] Enrichr library gene sets already downloaded in: /Users/victoriagao/.cache/gseapy, use local file
2023-09-27 16:28:14,887 [INFO] 0202 gene_sets have been filtered out when max_size=1000 and min_size=1
2023-09-27 16:28:14,888 [INFO] 0037 gene_sets used for further statistical testing.....
2023-09-27 16:28:14,888 [INFO] Start to run GSEA...Might take a while..................
2023-09-27 16:28:14,901 [INFO] Congratulations. GSEApy runs successfully.........

...

success for region13 VS 17
success for region13 VS 18
success for region13 VS 19
success for region13 VS 20
success for region14 VS 15
Error for column region14 VS 16: 
success for region14 VS 17
Error for column region14 VS 18: 
Error for column region14 VS 19: 
Error for column region14 VS 20: 
success for region15 VS 16


2023-09-27 16:28:22,129 [INFO] Enrichr library gene sets already downloaded in: /Users/victoriagao/.cache/gseapy, use local file
2023-09-27 16:28:22,134 [INFO] 0204 gene_sets have been filtered out when max_size=1000 and min_size=1
2023-09-27 16:28:22,135 [INFO] 0035 gene_sets used for further statistical testing.....
2023-09-27 16:28:22,135 [INFO] Start to run GSEA...Might take a while..................
2023-09-27 16:28:22,148 [INFO] Congratulations. GSEApy runs successfully................

2023-09-27 16:28:22,153 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!
2023-09-27 16:28:22,156 [WARNING] Duplicated values found in preranked stats: 75.00% of genes
The order of those genes will be arbitrary, which may produce unexpected results.
2023-09-27 16:28:22,156 [INFO] Parsing data files for GSEA.............................
2023-09-27 16:28:22,157 [INFO] Enrichr library gene sets already downloaded in: /Users/victoriagao/.cache/gseapy, use lo

success for region15 VS 17
success for region15 VS 18
success for region15 VS 19
success for region15 VS 20
success for region16 VS 17
Error for column region16 VS 18: 
Error for column region16 VS 19: 
Error for column region16 VS 20: 
success for region17 VS 18
success for region17 VS 19


2023-09-27 16:28:22,361 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!
2023-09-27 16:28:22,366 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!
2023-09-27 16:28:22,371 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!


success for region17 VS 20
Error for column region18 VS 19: 
Error for column region18 VS 20: 
Error for column region19 VS 20: 
